In [1]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv, find_dotenv
import congress
import agent

genai.__version__

'1.7.0'

Uncomment this if running in kaggle and make sureyou have secrets for the GOOGLE_API_KEY (available [here](https://aistudio.google.com/app/apikey)) and CONGRESS_API KEY (available [here](https://api.congress.gov/sign-up/)).
If you are running locally make sure you have those keys set in your .env

In [2]:
#from kaggle_secrets import UserSecretsClient
#os.environ["GOOGLE_API_KEY"] = UserSecretsClient().get_secret("GOOGLE_API_KEY")
#os.environ["CONGRESS_API_KEY"] = UserSecretsClient().get_secret("CONGRESS_API_KEY")

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
a = agent.CongressAgent()

In [5]:
a.run()

--> who represents NC in congress?


-- calling get_members(**{'state': 'NC'})
Here are the members of congress who represent North Carolina:  Ted Budd, Thomas Tillis, Addison McDowell, Tim Moore, Brad Knott, Mark Harris, Pat Harrigan, Valerie P. Foushee, Chuck Edwards, David Rouzer, Alma S. Adams, Richard Hudson, Virginia Foxx, Donald G. Davis, Deborah K. Ross, and Gregory F. Murphy.


--> Tell me about Deborah Ross


-- calling call_endpoint(**{'endpoint': 'https://api.congress.gov/v3/member/R000305?format=json'})
Deborah K. Ross is a Representative for the 2nd district of North Carolina. She is a Democrat and her current term started in 2021.


--> what bills has she sponsored?


...

--> what district does Ross reprsent?


HR 9720: AI Incident Reporting and Security Enhancement Act summary: This bill directs NIST to initiate a process to update processes and procedures for the voluntary tracking and processing of security and safety incidents associated with artificial intelligence. The latest action was Ordered to be Reported by Voice Vote.

HR 9737: Secure A.I. Act of 2024 summary: This bill directs federal agencies to establish mechanisms to manage security and safety risks related to the use of artificial intelligence (AI). The bill directs the National Institute of Standards and Technology (NIST) and the Cybersecurity and Infrastructure Agency (CISA) to develop a database to track AI-related safety and security incidents. The latest action was Referred to the Subcommittee on Cybersecurity and Infrastructure Protection.

HR 5576: Protect Working Musicians Act of 2023: There are no summaries available. The latest action was Referred to the House Committee on the Judiciary.

Representative Ross represe

--> do you have any information about that district?


HR 9720, the AI Incident Reporting and Security Enhancement Act, was ordered to be Reported by Voice Vote. HR9737, the Secure A.I. Act of 2024, was referred to the Subcommittee on Cybersecurity and Infrastructure Protection. HR5576, the Protect Working Musicians Act of 2023, was referred to the House Committee on the Judiciary.

Representative Ross represents district 2. I do not have any specific information about that district other than its number and that it is in North Carolina.



--> q


In [6]:
import requests

In [25]:
requests.get("https://www.govinfo.gov/bulkdata/json/BILLS/118/2/hr", headers={"accept": "application/json"}).json()

{'files': [{'mimeType': 'application/xml',
   'size': 12049,
   'formattedLastModifiedTime': '25-Mar-2025 15:03',
   'name': 'BILLS-118hr187rh.xml',
   'folder': False,
   'displayLabel': 'BILLS-118hr187rh.xml',
   'formattedSize': '11.8 kB',
   'link': 'https://www.govinfo.gov/bulkdata/BILLS/118/2/hr/BILLS-118hr187rh.xml',
   'justFileName': 'BILLS-118hr187rh.xml',
   'fileExtension': 'xml'},
  {'mimeType': 'application/xml',
   'size': 3564,
   'formattedLastModifiedTime': '10-Jan-2024 07:18',
   'name': 'BILLS-118hr6918ih.xml',
   'folder': False,
   'displayLabel': 'BILLS-118hr6918ih.xml',
   'formattedSize': '3.5 kB',
   'link': 'https://www.govinfo.gov/bulkdata/BILLS/118/2/hr/BILLS-118hr6918ih.xml',
   'justFileName': 'BILLS-118hr6918ih.xml',
   'fileExtension': 'xml'},
  {'mimeType': 'application/xml',
   'size': 6447,
   'formattedLastModifiedTime': '10-Jan-2024 07:18',
   'name': 'BILLS-118hr6370rh.xml',
   'folder': False,
   'displayLabel': 'BILLS-118hr6370rh.xml',
   'forma

In [28]:
r = requests.get('https://www.govinfo.gov/bulkdata/BILLS/118/2/hr/BILLS-118hr187rh.xml')

In [31]:
import xml.etree.ElementTree as ET
tree = ET.fromstring(r.text)

In [18]:
import chromadb

In [20]:
c = chromadb.Client()

In [21]:
help(c)

Help on Client in module chromadb.api.client object:

class Client(chromadb.api.shared_system_client.SharedSystemClient, chromadb.api.ClientAPI)
 |  Client(tenant: str = 'default_tenant', database: str = 'default_database', settings: chromadb.config.Settings = Settings(environment='', chroma_api_impl='chromadb.api.rust.RustBindingsAPI', chroma_server_nofile=None, chroma_server_thread_pool_size=40, tenant_id='default', topic_namespace='default', chroma_server_host=None, chroma_server_headers=None, chroma_server_http_port=None, chroma_server_ssl_enabled=False, chroma_server_ssl_verify=None, chroma_server_api_default_path=<APIVersion.V2: '/api/v2'>, chroma_server_cors_allow_origins=[], is_persistent=False, persist_directory='./chroma', chroma_memory_limit_bytes=0, chroma_segment_cache_policy=None, allow_reset=False, chroma_auth_token_transport_header=None, chroma_client_auth_provider=None, chroma_client_auth_credentials=None, chroma_server_auth_ignore_paths={'/api/v2': ['GET'], '/api/v2/h